# Streamlit 应用：基于 Unsplash API 的图片搜索与颜色聚类

本项目演示如何使用 Streamlit 构建一个网页应用，利用 Unsplash API 搜索图片，并通过 K-Means 聚类对图片的主要颜色进行二次筛选。

主要功能包括：
1. 自定义 CSS 样式，优化侧边栏和按钮的显示；
2. 使用 Unsplash API 搜索图片，并获取 'regular' 尺寸的图片 URL；
3. 侧边栏提供关键词和颜色选择，支持用户输入查询条件；
4. 可选的 K-Means 聚类用于根据颜色对搜索结果进行二次筛选；
5. 显示图片，并提供“Load More”按钮加载更多图片。

下面的 Notebook 包含完整的代码和详细注释，帮助理解各部分代码的用途及其原理。

In [ ]:
import streamlit as st
import requests
import numpy as np
import cv2
from sklearn.cluster import KMeans
from skimage import color
from io import BytesIO
from PIL import Image

# 反思：导入相关库，其中 streamlit 用于构建应用界面，requests 用于网络请求，cv2 和 KMeans 及 skimage 用于图像处理和颜色聚类。

## 1. 自定义 CSS 样式

以下代码通过 `st.markdown` 注入自定义 CSS，调整侧边栏、按钮、标题等的样式，提升界面美观度。

In [ ]:
st.markdown("""
    <style>
    /* 给左侧Sidebar增加背景色与轻微阴影，以增强层次感 */
    [data-testid="stSidebar"] {
        background-color: #f8f9fa;
        box-shadow: 2px 0 5px rgba(0, 0, 0, 0.1);
    }

    /* 主标题样式：加大字号、字母间距、减少底部外边距 */
    .main-title {
        font-size: 3rem;
        letter-spacing: 2px;
        font-weight: 700;
        margin-bottom: 0.5rem;
        text-align: center;
    }

    /* 侧边栏标题（Tone Selector）的样式 */
    [data-testid="stSidebar"] h2 {
        font-size: 1.25rem;
        font-weight: 700;
        margin-bottom: 0.5rem;
        letter-spacing: 1px;
    }

    /* 下拉框与按钮之间的默认间距 */
    .stSelectbox, .stButton {
        margin-bottom: 1rem;
    }

    /* 优化按钮外观与尺寸，使用更柔和的灰色 */
    .stButton > button {
        font-size: 16px;
        padding: 0.5rem 1.5rem;
        border-radius: 4px;
        background-color: #cccccc;
        color: #333;
        border: none;
        transition: background-color 0.3s ease;
    }
    .stButton > button:hover {
        background-color: #bbbbbb;
    }
    </style>
""", unsafe_allow_html=True)

# 反思：自定义 CSS 有助于统一界面风格，提升用户体验。

## 2. 自定义主标题

使用 HTML 标签和 CSS 类为主界面添加一个自定义样式的主标题。

In [ ]:
st.markdown("<h1 class='main-title'>SplashPatterns</h1>", unsafe_allow_html=True)

# 反思：主标题可以吸引用户注意，同时传递应用的核心功能。

## 3. Unsplash API 配置及图片搜索函数

设置 Unsplash API 的 Access Key 及搜索 URL，并定义函数 `search_unsplash_single_page` 用于搜索单页图片，返回 'regular' 尺寸的图片 URL 列表。

In [ ]:
UNSPLASH_ACCESS_KEY = "cV2Slomwnm9YY0dp_lRF40J2QGatJfmmPxwJyzyZIlA"  # 请替换为你自己的 Access Key
UNSPLASH_SEARCH_URL = "https://api.unsplash.com/search/photos"

def search_unsplash_single_page(query, per_page=24, page=1, color=None):
    """
    在 Unsplash 上搜索单页图片，返回 'regular' 图片 URL 列表。
    如果传入 color 参数，则会使用 Unsplash 的内置 color 筛选。
    """
    headers = {"Authorization": f"Client-ID {UNSPLASH_ACCESS_KEY}"}
    params = {
        "query": query,
        "page": page,
        "per_page": per_page
    }
    if color:
        params["color"] = color

    try:
        response = requests.get(UNSPLASH_SEARCH_URL, headers=headers, params=params)
        if response.status_code != 200:
            st.warning(f"Unsplash API 返回错误: {response.status_code}")
            return []
        data = response.json()
    except Exception as e:
        st.error(f"请求 Unsplash API 出错: {e}")
        return []

    image_urls = []
    for item in data.get("results", []):
        url = item.get("urls", {}).get("regular")
        if url:
            image_urls.append(url)
    return image_urls

# 反思：该函数封装了对 Unsplash API 的请求，便于后续批量调用以获取搜索结果。

## 4. 侧边栏输入与 Session State 初始化

使用 Streamlit 的侧边栏创建输入控件，让用户输入搜索关键词和选择颜色，同时初始化 Session State 用于保存图片 URL 列表和当前页码。

In [ ]:
st.sidebar.header("Tone Selector")

# 关键词输入框
keyword = st.sidebar.text_input("Search Keyword")

# 颜色选择（Unsplash color 参数支持的值）
colors = [
    "None", "black_and_white", "black", "white", "yellow", "orange",
    "red", "purple", "magenta", "green", "teal", "blue"
]
selected_color = st.sidebar.selectbox("Select a color (Unsplash filter)", colors)

# 可选：勾选后前端使用 K-Means 进行二次颜色聚类
refine_color = st.sidebar.checkbox("Refine with K-Means")

# 初始化 Session State
if "images" not in st.session_state:
    st.session_state["images"] = []
if "page" not in st.session_state:
    st.session_state["page"] = 1

# 反思：侧边栏提供交互式输入，使用户能动态调整搜索条件。Session State 用于在页面间保存数据。

## 5. 加载图片函数

定义函数 `load_images`，根据当前页面编号调用 Unsplash API，并将返回的图片 URL 追加到 Session State 中。

In [ ]:
def load_images(query, color=None):
    """
    根据 st.session_state["page"] 调用 API，获取 24 张图片，并追加到 st.session_state["images"] 中。
    """
    new_urls = search_unsplash_single_page(
        query,
        per_page=24,
        page=st.session_state["page"],
        color=color
    )
    st.session_state["images"].extend(new_urls)

# 反思：该函数便于按页加载图片，适用于需要分页加载的场景。

## 6. 点击 "Search" 按钮后的逻辑

当用户点击侧边栏的 "Search" 按钮时，重置 Session State，然后加载前 3 页图片。

In [ ]:
if st.sidebar.button("Search"):
    # 每次新搜索时重置页码和图片列表
    st.session_state["page"] = 1
    st.session_state["images"] = []
    
    # 处理颜色参数
    c = None if selected_color == "None" else selected_color
    
    # 假设加载前 3 页图片
    for page_idx in range(1, 4):
        st.session_state["page"] = page_idx
        load_images(keyword, color=c)
    
    # 反思：加载多页数据后，st.session_state["images"] 存储了较多图片 URL，可用于后续展示。

## 7. （可选）使用 K-Means 进行二次色彩聚类筛选

定义几个函数：
1. `get_dominant_color`：利用 K-Means 聚类提取图片的主要颜色；
2. `rgb_to_lab`：将 RGB 色值转换为 Lab 色彩空间；
3. `color_distance`：计算两种 Lab 颜色的欧氏距离。

反思：通过对图片进行颜色聚类，我们可以根据用户选择的颜色进一步筛选图片，提升搜索结果的相关性。

In [ ]:
def get_dominant_color(img, k=3):
    """使用 K-Means 从图片提取最主要的颜色（返回值为 (r, g, b)，取 0-1 区间）。"""
    # 将图片转换为 NumPy 数组并缩小尺寸，降低计算量
    img = np.array(img.resize((100, 100)))
    img = img.reshape(-1, 3).astype(np.float32) / 255.0

    km = KMeans(n_clusters=k, random_state=42).fit(img)
    labels, counts = np.unique(km.labels_, return_counts=True)
    major_cluster = labels[np.argmax(counts)]
    dominant_rgb = km.cluster_centers_[major_cluster]
    return dominant_rgb  

def rgb_to_lab(rgb):
    """将 (r, g, b)（0-1）转换为 Lab 色彩空间"""
    arr = np.array(rgb).reshape(1, 1, 3)
    lab = color.rgb2lab(arr)
    return lab[0, 0, :]

def color_distance(c1, c2):
    """计算两种 Lab 颜色之间的欧氏距离"""
    return np.sqrt(np.sum((c1 - c2) ** 2))

# 反思：以上函数封装了颜色提取和比较的逻辑，为后续的图片二次筛选提供支持。

## 8. 显示图片和 “Load More” 按钮

如果 Session State 中有图片，则以网格方式显示，并提供“Load More”按钮，点击后加载更多图片。

反思：通过分页加载和动态显示图片，应用可以在保证响应速度的同时，展示更多搜索结果。

In [ ]:
if st.session_state["images"]:
    # 如果需要二次筛选，则进行 K-Means 色彩聚类筛选
    if refine_color and selected_color != "None":
        # 定义一个简单的颜色映射字典（可根据需要调整）
        color_map = {
            "black": (0, 0, 0),
            "white": (1, 1, 1),
            "yellow": (1, 1, 0),
            "orange": (1, 0.65, 0),
            "red": (1, 0, 0),
            "purple": (0.5, 0, 0.5),
            "magenta": (1, 0, 1),
            "green": (0, 1, 0),
            "teal": (0, 0.5, 0.5),
            "blue": (0, 0, 1),
            "black_and_white": (0.5, 0.5, 0.5)
        }
        target_rgb = color_map.get(selected_color, (0.5, 0.5, 0.5))
        target_lab = rgb_to_lab(target_rgb)

        refined_data = []
        for url in st.session_state["images"]:
            try:
                resp = requests.get(url, timeout=5)
                if resp.status_code == 200:
                    img_pil = Image.open(BytesIO(resp.content)).convert("RGB")
                    dom_rgb = get_dominant_color(img_pil, k=3)
                    dist = color_distance(rgb_to_lab(dom_rgb), target_lab)
                    refined_data.append((url, dist))
            except:
                pass

        refined_data.sort(key=lambda x: x[1])
        top_n = 200
        refined_data = refined_data[:top_n]
        st.session_state["images"] = [x[0] for x in refined_data]

    # 网格显示图片
    num_cols = 4
    cols = st.columns(num_cols)
    for idx, url in enumerate(st.session_state["images"]):
        with cols[idx % num_cols]:
            st.image(url, use_container_width=True)

    # 提供 "Load More" 按钮，点击后加载下一页
    if st.button("Load More"):
        st.session_state["page"] += 1
        c = None if selected_color == "None" else selected_color
        load_images(keyword, color=c)
else:
    if st.session_state["page"] == 1:
        st.write("No images to display. Please click 'Search' on the sidebar.")

# 反思：这部分代码实现了图片的动态加载与展示，结合了分页和二次颜色筛选，能够提升用户搜索体验。

## 总结与反思

通过本项目，我们实现了一个基于 Streamlit 的网页应用，利用 Unsplash API 搜索图片，并通过 K-Means 聚类进行颜色筛选。主要收获包括：

- **前端样式定制**：通过自定义 CSS 提升了界面美观度与用户体验；
- **API 调用与数据处理**：封装 Unsplash API 请求，使代码模块化；
- **颜色聚类与二次筛选**：使用 K-Means 聚类提取图片主要颜色，并根据目标颜色排序筛选，体现了计算机视觉在实际应用中的价值；
- **分页加载**：利用 Session State 保存数据，实现分页加载，保证界面响应速度。

这些技术和方法为我们在实际开发中构建交互式数据应用提供了有力支持。希望你能从中学习到模块化编程、API 调用、图像处理以及前端样式定制等多方面知识。